In [1]:
# This default renderer is used for sphinx docs only. Please delete this cell in IPython.
import plotly.io as pio
pio.renderers.default = "png"

In [2]:

import os
import torch
import unittest
import numpy as np
from PIL import Image as PilImage
from omnixai.data.text import Text
from omnixai.data.image import Image
from omnixai.data.multi_inputs import MultiInputs
from omnixai.preprocessing.image import Resize
from omnixai.explainers.vision_language.specific.gradcam import GradCAM

from lavis.models import BlipITM
from lavis.processors import load_processor

In [17]:
from lavis.models import load_model_and_preprocess

In [9]:
from lavis.models import load_model

In [3]:
image = Resize(size=480).transform(
    Image(PilImage.open("/home/xtest/lavis/LAVIS/cache/heat/images/test/7099.jpg").convert("RGB")))
text = Text("a pedestrian crosses the road in front of a line of cars.")
inputs = MultiInputs(image=image, text=text)

In [5]:
ckp_path = \
    "/home/xtest/lavis/LAVIS/lavis/output/BLIP/Retrieval_COCO/20221129121/checkpoint_best.pth"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# model = load_model(name="blip_caption", model_type="base_coco", is_eval=True, device=device, checkpoint=ckp_path)

In [10]:
# model = load_model(
#         name="blip_image_text_matching", model_type="base", is_eval=True, device=device, checkpoint=ckp_path
#     )

In [18]:
model, image_processor, text_processors = load_model_and_preprocess("blip_image_text_matching", "base", device=device, is_eval=True)

In [19]:
# model = BlipITM(pretrained=pretrained_path, vit="base")
# image_processor = load_processor("blip_image_eval").build(image_size=384)
# text_processor = load_processor("blip_caption")
tokenizer = BlipITM.init_tokenizer()

In [20]:
def preprocess(x: MultiInputs):
    images = torch.stack([image_processor(z.to_pil()) for z in x.image])
    texts = [text_processor(z) for z in x.text.values]
    return images, texts

In [22]:
for el in model.text_encoder.base_model.base_model.encoder.layer[6].crossattention:
    print(el)

TypeError: 'BertAttention' object is not iterable

In [21]:
explainer = GradCAM(
    model=model,
    target_layer=model.text_encoder.base_model.base_model.encoder.layer[6].
        crossattention.self.attention_probs_layer,
    preprocess_function=preprocess,
    tokenizer=tokenizer,
    loss_function=lambda outputs: outputs[:, 1].sum()
)

AttributeError: 'BertSelfAttention' object has no attribute 'attention_probs_layer'

In [15]:
pretrained_path = \
    "https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth"
model = BlipITM(pretrained=pretrained_path, vit="base")
image_processor = load_processor("blip_image_eval").build(image_size=384)
text_processor = load_processor("blip_caption")
tokenizer = BlipITM.init_tokenizer()

TypeError: __init__() got an unexpected keyword argument 'pretrained'